In [2]:
import os
import gdal
import scipy.ndimage
import shapefile
from pyproj import Proj, transform

Define a function to create a projection (.prj) file.

In [3]:
def getWKT_PRJ (epsg_code):
    import urllib
    wkt = urllib.urlopen("http://spatialreference.org/ref/epsg/{0}/prettywkt/".format(epsg_code))
    remove_spaces = wkt.read().replace(" ","")
    output = remove_spaces.replace("\n", "")
    return output

Defining path name to the shape file

In [4]:
shp_folder = "/Users/rohankumar/espc2017_op_cafe/shp/"

Using PyShp create a Reader object to access the data from the polygonized_Stuff Shapefile.

In [5]:
shpf = shapefile.Reader(shp_folder + "polygonized_Stuff.shp")

Create a Writer object to write data to as a new Shapefile.

In [6]:
wgs_shp = shapefile.Writer("/Users/rohankumar/espc2017_op_cafe/shapefile.POLYGON")

Set variables for access to the field information of both the original and new Shapefile.

In [7]:
fields = shpf.fields
wgs_fields = wgs_shp.fields

We will grab all the field info from the original and copy it into the new. The ‘Deletion Flag’ as set in the Shapefile standard will be passed over (the tuple in the if statement), and we want data from the lists that follow the tuple that define the field name, data type and field length. Basically we are simply replicating the field structure from the original into the new.

In [8]:
for name in fields:
    if type(name) == "tuple":
        continue
    else:
        args = name
        wgs_shp.field(*args)

Now we want to populate the fields with attribute information. Create a variable to access the records of the original file.

In [9]:
records = shpf.records()

Copy the records from the original into the new.

In [10]:
for row in records:
    args = row
    wgs_shp.record(*args)

In the above snippet the args variable holds each record as a list and then unpacks that list as arguments in wgs_shp.record(attr_1, attr_2, attr_3….), which creates a record in the dbf file.

We now have all the attribute data copied over. Let’s begin the quest to convert the data from ITM to WGS84! Define the input projection (the projection of the original file), and an output projection using PyProj..

In [14]:
input_projection = Proj(init="epsg:4326")
output_projection = Proj(init="epsg:3395")

We need to access the geometry of the features in the original file so give yourself access to it.

In [15]:
geom = shpf.shapes()

Now we loop through each feature in the original dataset, access every point that makes up the geometry, convert the coordinates for each point and re-assemble transformed geometry in the new Shapefile. The if statement will handle geometry with only one part making up the feature.

In [16]:
for feature in geom:
    # if there is only one part
    if len(feature.parts) == 1:
        # create empty list to store all the coordinates
        poly_list = []

        # get each coord that makes up the polygon
        for coords in feature.points:
               x, y = coords[0], coords[1]
               # tranform the coord
               new_x, new_y = transform(input_projection, output_projection, x, y)
               # put the coord into a list structure
               poly_coord = [float(new_x), float(new_y)]
               # append the coords to the polygon list
               poly_list.append(poly_coord)
           # add the geometry to the shapefile.
        wgs_shp.parts=poly_list
    else:
        # append the total amount of points to the end of the parts list
        feature.parts.append(len(feature.points))
        # enpty list to store all the parts that make up the complete feature
        poly_list = []
        # keep track of the part being added
        parts_counter = 0

        # while the parts_counter is less than the amount of parts
        while parts_counter < len(feature.parts) - 1:
            # keep track of the amount of points added to the feature
            coord_count = feature.parts[parts_counter]
            # number of points in each part
            no_of_points = abs(feature.parts[parts_counter] - feature.parts[parts_counter + 1])
            # create list to hold individual parts - these get added to poly_list[]
            part_list = []
            # cut off point for each part
            end_point = coord_count + no_of_points

            # loop through each part
            while coord_count < end_point:
                for coords in feature.points[coord_count:end_point]:
                    x, y = coords[0], coords[1]
                    # tranform the coord
                    new_x, new_y = transform(input_projection, output_projection, x, y)
                    # put the coord into a list structure
                    poly_coord = [float(new_x), float(new_y)]
                    # append the coords to the part list
                    part_list.append(poly_coord)
                    coord_count = coord_count + 1
        # append the part to the poly_list
        poly_list.append(part_list)
        parts_counter = parts_counter + 1
    # add the geometry to to new file
wgs_shp.parts=poly_list

RuntimeError: b'latitude or longitude exceeded limits'

Save the shapefile

In [ ]:
wgs_shp.save(shp_folder + "Polygonized_Stuff_wgs.shp")

In [ ]:
prj = open(shp_folder + "Polygonized_Stuff_wgs.prj", "w")
epsg = getWKT_PRJ("3395")
prj.write(epsg)
prj.close()

In [ ]:
prj